In [ ]:
# Install Dependencies

!pip uninstall -y numpy
!pip install --force-reinstall numpy==1.26.4

!pip install torch==2.2.1 torchvision==0.17.1 torchaudio==2.2.1 sentence-transformers==2.2.2
!pip install pandas==2.0.0
!pip install transformers==4.41.0 scikit-learn==1.2.0
!pip install huggingface-hub==0.25.2
!pip install nltk==3.8.1 rouge-score==0.1.2 bert-score==0.3.13 -q
!pip install tqdm==4.66.5 -q
!pip install shap==0.46.0 lime==0.2.0.1

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.

In [ ]:
# Setup and Imports

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer,
    DPRContextEncoder, DPRQuestionEncoder,
    DPRContextEncoderTokenizer, DPRQuestionEncoderTokenizer
)
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
import os
from google.colab import drive
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import bert_score
import shap
import lime
import lime.lime_text
import string
import nltk
import json

nltk.download('wordnet')
nltk.download('punkt')

# Mount Google Drive
drive.mount('/content/drive')

# Configuration (aligned with previous steps)
class Config:
    BASE_PATH = "/content/drive/MyDrive/LJMU-Datasets"
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    T5_MODEL_NAME = "t5-small"
    DPR_CTX_MODEL_NAME = "facebook/dpr-ctx_encoder-single-nq-base"
    DPR_QUESTION_MODEL_NAME = "facebook/dpr-question_encoder-single-nq-base"
    BATCH_SIZE = 16
    MAX_EPOCHS = 3
    NUM_WORKERS = 4
    MAX_LENGTH = 256
    SUBSET_SIZE = 500
    HOTPOTQA_MAX_SAMPLES = 1000
    WIKIDATA_SUBSET_SIZE = 30000

CONFIG = Config()
print(f"Using device: {CONFIG.DEVICE}")

# Clear GPU memory
torch.cuda.empty_cache()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Mounted at /content/drive
Using device: cuda


In [ ]:
# Load Artifacts from Step 3 and Rebuild DataLoaders

save_path = '/content/drive/MyDrive/bert_retrieval_artifacts'

# Load raw datasets
qa_train_path = os.path.join(CONFIG.BASE_PATH, "qa_train.csv")
qa_val_path = os.path.join(CONFIG.BASE_PATH, "qa_val.csv")
triple_train_path = os.path.join(CONFIG.BASE_PATH, "triple_train.csv")

qa_train_df = pd.read_csv(qa_train_path)
qa_val_df = pd.read_csv(qa_val_path)
triple_train_df = pd.read_csv(triple_train_path)

# Split triple_train_df into train and validation sets (80/20)
triple_train_df, triple_val_df = train_test_split(triple_train_df, train_size=0.8, random_state=42)
print(f"Triple Train Size: {len(triple_train_df)}, Triple Val Size: {len(triple_val_df)}")

# Load all_candidates
with open(os.path.join(save_path, 'all_candidates.pkl'), 'rb') as f:
    import pickle
    all_candidates = pickle.load(f)

# Define RetrievalDataset class
class RetrievalDataset(Dataset):
    def __init__(self, df: pd.DataFrame, t5_tokenizer: T5Tokenizer, dpr_question_tokenizer: DPRQuestionEncoderTokenizer,
                 max_length: int = 256, task: str = "qa", candidate_objects: list = None):
        self.t5_tokenizer = t5_tokenizer
        self.dpr_question_tokenizer = dpr_question_tokenizer
        self.max_length = max_length
        self.task = task
        self.data = df
        self.candidate_objects = candidate_objects

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        question = row["question"]
        context = row["context"]
        answer = row["answer"]

        if self.task == "qa":
            t5_input_text = f"answer the question exactly as in the reference: {question} context: {context} ->"
        else:
            t5_input_text = f"complete the triple with the exact object: {question} ->"
        t5_inputs = self.t5_tokenizer(
            t5_input_text,
            return_tensors="pt",
            max_length=self.max_length,
            truncation=True,
            padding="max_length"
        )
        t5_labels = self.t5_tokenizer(
            answer,
            return_tensors="pt",
            max_length=self.max_length,
            truncation=True,
            padding="max_length"
        )

        dpr_inputs = self.dpr_question_tokenizer(
            question,
            return_tensors="pt",
            max_length=self.max_length,
            truncation=True,
            padding="max_length"
        )

        item = {
            "task": self.task,
            "t5_input_ids": t5_inputs["input_ids"].squeeze(),
            "t5_attention_mask": t5_inputs["attention_mask"].squeeze(),
            "t5_labels": t5_labels["input_ids"].squeeze(),
            "dpr_input_ids": dpr_inputs["input_ids"].squeeze(),
            "dpr_attention_mask": dpr_inputs["attention_mask"].squeeze(),
            "question": question,
            "answer": answer
        }

        if self.task == "triple" and self.candidate_objects:
            label_idx = self.candidate_objects.index(answer) if answer in self.candidate_objects else -1
            item["label_idx"] = label_idx

        return item

# Create DataLoaders from raw data
t5_tokenizer = T5Tokenizer.from_pretrained(CONFIG.T5_MODEL_NAME)
dpr_question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained(CONFIG.DPR_QUESTION_MODEL_NAME)

qa_train_dataset = RetrievalDataset(qa_train_df, t5_tokenizer, dpr_question_tokenizer, task="qa", candidate_objects=all_candidates)
qa_val_dataset = RetrievalDataset(qa_val_df, t5_tokenizer, dpr_question_tokenizer, task="qa", candidate_objects=all_candidates)
triple_train_dataset = RetrievalDataset(triple_train_df, t5_tokenizer, dpr_question_tokenizer, task="triple", candidate_objects=all_candidates)
triple_val_dataset = RetrievalDataset(triple_val_df, t5_tokenizer, dpr_question_tokenizer, task="triple", candidate_objects=all_candidates)

qa_train_loader = DataLoader(qa_train_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=True, num_workers=CONFIG.NUM_WORKERS)
qa_val_loader = DataLoader(qa_val_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False, num_workers=CONFIG.NUM_WORKERS)
triple_train_loader = DataLoader(triple_train_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=True, num_workers=CONFIG.NUM_WORKERS)
triple_val_loader = DataLoader(triple_val_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False, num_workers=CONFIG.NUM_WORKERS)

print(f"Created QA DataLoaders: QA Train={len(qa_train_dataset)}, QA Val={len(qa_val_dataset)}")
print(f"Created Triple DataLoaders: Triple Train={len(triple_train_dataset)}, Triple Val={len(triple_val_dataset)}")

# Load T5 model and tokenizer
t5_qa_model = T5ForConditionalGeneration.from_pretrained(CONFIG.T5_MODEL_NAME).to(CONFIG.DEVICE)
t5_qa_model.load_state_dict(torch.load(os.path.join(save_path, 't5_qa_model.pt')))
t5_tokenizer = T5Tokenizer.from_pretrained(CONFIG.T5_MODEL_NAME)
t5_qa_model.eval()

# Load DPR models and tokenizers (post-RL fine-tuning)
ctx_encoder = DPRContextEncoder.from_pretrained(os.path.join(CONFIG.BASE_PATH, "dpr_ctx_encoder_rl")).to(CONFIG.DEVICE)
question_encoder = DPRQuestionEncoder.from_pretrained(os.path.join(CONFIG.BASE_PATH, "dpr_question_encoder_rl")).to(CONFIG.DEVICE)
ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained(CONFIG.DPR_CTX_MODEL_NAME)
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained(CONFIG.DPR_QUESTION_MODEL_NAME)
candidate_embeddings = torch.load(os.path.join(save_path, 'dpr_candidate_embeddings.pt')).to(CONFIG.DEVICE)

# Load sentence transformer
sentence_transformer = SentenceTransformer('all-MiniLM-L6-v2')

print("Artifacts loaded from Step 3 and DataLoaders rebuilt.")

Triple Train Size: 3999, Triple Val Size: 1000


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Artifacts loaded from Step 3 and DataLoaders rebuilt.


In [ ]:
# Define Helper Functions

# Normalize text for evaluation
def normalize_text(text: str) -> str:
    text = str(text).lower().strip()
    text = text.translate(str.maketrans("", "", string.punctuation))
    articles = {'a', 'an', 'the'}
    words = text.split()
    words = [word for word in words if word not in articles]
    return ' '.join(words)

# Compute BLEU score
def compute_bleu(generated: str, reference: str) -> float:
    return sentence_bleu([reference.split()], generated.split())

# Compute ROUGE-L score
def compute_rouge_l(generated: str, reference: str) -> float:
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    return scorer.score(reference, generated)['rougeL'].fmeasure

# Compute BERTScore
def compute_bertscore(generated: str, reference: str) -> float:
    return bert_score.score([generated], [reference], lang="en", verbose=False)[2].mean().item()

print("Helper functions defined.")

Helper functions defined.


In [ ]:
# Explainability with LIME and Custom Permutation SHAP

# Function to compute DPR similarity scores for a given question embedding and candidates
def compute_dpr_similarities_from_embedding(question_embedding, candidates, question_encoder, ctx_encoder, tokenizer):
    with torch.no_grad():
        # Convert NumPy array to PyTorch tensor and ensure correct device and shape
        question_embedding = torch.tensor(question_embedding, dtype=torch.float32).to(CONFIG.DEVICE).unsqueeze(0)  # Shape: (1, 768)

        # Encode candidates using DPR ctx_encoder
        candidate_inputs = tokenizer(
            candidates,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=CONFIG.MAX_LENGTH
        )
        candidate_inputs = {k: v.to(CONFIG.DEVICE) for k, v in candidate_inputs.items()}
        candidate_embeddings = ctx_encoder(**candidate_inputs).pooler_output  # Shape: (num_candidates, 768)

        # Compute similarities
        similarities = torch.matmul(question_embedding, candidate_embeddings.T).squeeze(0)  # Shape: (num_candidates,)
        return similarities.cpu().numpy()

# Function to convert text to embeddings using SentenceTransformer and align with DPR
def text_to_dpr_embedding(texts, sentence_transformer, question_encoder, tokenizer):
    if isinstance(texts, str):
        texts = [texts]
    # Convert to DPR embeddings directly
    dpr_inputs = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=CONFIG.MAX_LENGTH
    )
    dpr_inputs = {k: v.to(CONFIG.DEVICE) for k, v in dpr_inputs.items()}
    with torch.no_grad():
        dpr_embeddings = question_encoder(**dpr_inputs).pooler_output  # Shape: (n_texts, 768)
    return dpr_embeddings.cpu().numpy()

# Wrapper function for general use
def dpr_predict(texts):
    embeddings = text_to_dpr_embedding(texts, sentence_transformer, question_encoder, question_tokenizer)
    similarities_list = []
    for emb in embeddings:
        similarities = compute_dpr_similarities_from_embedding(emb, all_candidates[:100], question_encoder, ctx_encoder, question_tokenizer)
        similarities_list.append(similarities)
    return np.array(similarities_list)

# Wrapper function for LIME to handle perturbed samples
def lime_dpr_predict(texts):
    # Ensure texts is a list of strings
    if not isinstance(texts, list):
        texts = [texts]
    # Get the number of perturbed samples from the input (LIME provides this)
    num_samples = len(texts)
    embeddings = text_to_dpr_embedding(texts, sentence_transformer, question_encoder, question_tokenizer)  # Shape: (num_samples, 768)
    similarities_list = []
    for emb in embeddings:
        similarities = compute_dpr_similarities_from_embedding(emb, all_candidates[:100], question_encoder, ctx_encoder, question_tokenizer)
        similarities_list.append(similarities)
    # Ensure output shape matches LIME's expectation (num_samples, num_classes)
    similarities_array = np.array(similarities_list)  # Shape: (num_samples, num_candidates)
    # Convert to binary classification (similar vs. not similar) for LIME
    probs = (similarities_array - similarities_array.min()) / (similarities_array.max() - similarities_array.min())
    return np.vstack([1 - probs, probs]).T  # Shape: (num_samples, 2)

# Custom permutation-based SHAP approximation
def custom_permutation_shap(question, candidates, num_permutations=50):
    baseline_similarities = dpr_predict([question])[0]
    feature_importance = np.zeros(len(candidates))
    original_idx = np.argmax(baseline_similarities)

    for _ in range(num_permutations):
        # Permute the candidates
        permuted_candidates = candidates.copy()
        np.random.shuffle(permuted_candidates)
        permuted_similarities = dpr_predict([question])[0]  # Recompute with original question
        permuted_idx = np.argmax(permuted_similarities)
        importance = abs(baseline_similarities[original_idx] - permuted_similarities[permuted_idx])
        feature_importance += importance
    return feature_importance / num_permutations

# Select examples for explainability (10 QA, 10 triple)
qa_examples = qa_val_df.sample(n=10, random_state=42)
triple_examples = triple_val_df.sample(n=10, random_state=42)

# LIME Explainability (Primary Method)
print("Computing LIME explanations...")
lime_explainer = lime.lime_text.LimeTextExplainer(class_names=["similarity_score"])

lime_explanations_qa = []
lime_explanations_triple = []

# LIME for QA examples
for i, row in qa_examples.iterrows():
    question = row["question"]
    explanation = lime_explainer.explain_instance(question, lime_dpr_predict, num_features=5, num_samples=100)  # Match num_samples to LIME default
    lime_explanations_qa.append((question, explanation.as_list()))
    print(f"LIME computed for QA example {i+1}/10")

# LIME for triple examples
for i, row in triple_examples.iterrows():
    question = row["question"]
    explanation = lime_explainer.explain_instance(question, lime_dpr_predict, num_features=5, num_samples=100)
    lime_explanations_triple.append((question, explanation.as_list()))
    print(f"LIME computed for triple example {i+1}/10")

# Custom Permutation SHAP (Secondary Method)
print("Computing custom permutation SHAP explanations...")
permutation_shap_qa = []
permutation_shap_triple = []

# Permutation SHAP for QA examples
for i, row in qa_examples.iterrows():
    question = row["question"]
    importance = custom_permutation_shap(question, all_candidates[:100])
    permutation_shap_qa.append((question, importance))
    print(f"Permutation SHAP computed for QA example {i+1}/10")

# Permutation SHAP for triple examples
for i, row in triple_examples.iterrows():
    question = row["question"]
    importance = custom_permutation_shap(question, all_candidates[:100])
    permutation_shap_triple.append((question, importance))
    print(f"Permutation SHAP computed for triple example {i+1}/10")

# Save explainability results
explainability_results = {
    "lime_qa": lime_explanations_qa,
    "lime_triple": lime_explanations_triple,
    "permutation_shap_qa": [(q, v.tolist()) for q, v in permutation_shap_qa],
    "permutation_shap_triple": [(q, v.tolist()) for q, v in permutation_shap_triple]
}

explainability_path = os.path.join(CONFIG.BASE_PATH, "explainability_results.json")
with open(explainability_path, "w") as f:
    json.dump(explainability_results, f)
print(f"Saved explainability results at {explainability_path}")

# Print sample explanations
print("Sample LIME Explanations (QA):")
for question, explanation in lime_explanations_qa[:2]:
    print(f"Question: {question}")
    print(f"LIME Explanation: {explanation}\n")

print("Sample LIME Explanations (Triple):")
for question, explanation in lime_explanations_triple[:2]:
    print(f"Question: {question}")
    print(f"LIME Explanation: {explanation}\n")

print("Sample Permutation SHAP Explanations (QA):")
for question, importance in permutation_shap_qa[:2]:
    print(f"Question: {question}")
    print(f"Permutation SHAP Importance: {importance[:5]}...\n")

print("Sample Permutation SHAP Explanations (Triple):")
for question, importance in permutation_shap_triple[:2]:
    print(f"Question: {question}")
    print(f"Permutation SHAP Importance: {importance[:5]}...\n")

Computing LIME explanations...
LIME computed for QA example 84/10
LIME computed for QA example 54/10
LIME computed for QA example 71/10
LIME computed for QA example 46/10
LIME computed for QA example 45/10
LIME computed for QA example 40/10
LIME computed for QA example 23/10
LIME computed for QA example 81/10
LIME computed for QA example 11/10
LIME computed for QA example 1/10
LIME computed for triple example 1974/10
LIME computed for triple example 3707/10
LIME computed for triple example 528/10
LIME computed for triple example 535/10
LIME computed for triple example 2944/10
LIME computed for triple example 4776/10
LIME computed for triple example 599/10
LIME computed for triple example 1727/10
LIME computed for triple example 2226/10
LIME computed for triple example 1841/10
Computing custom permutation SHAP explanations...
Permutation SHAP computed for QA example 84/10
Permutation SHAP computed for QA example 54/10
Permutation SHAP computed for QA example 71/10
Permutation SHAP compu

In [ ]:
# Quantitative Validation (Metrics)

# Evaluate T5 QA on both QA and triple tasks
def evaluate_t5(model, val_loader, task: str = "qa"):
    print(f"Evaluating T5 for {task}...")
    model.eval()
    bleu_scores, rouge_scores, bert_scores = [], [], []
    sample_outputs = []
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Evaluating {task}"):
            input_ids = batch["t5_input_ids"].to(CONFIG.DEVICE)
            attention_mask = batch["t5_attention_mask"].to(CONFIG.DEVICE)
            references = batch["answer"]
            generated_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=100,
                num_beams=20,
                temperature=0.5,
                no_repeat_ngram_size=2
            )
            generated_texts = [t5_tokenizer.decode(g_ids, skip_special_tokens=True) for g_ids in generated_ids]
            for gen, ref in zip(generated_texts, references):
                gen = normalize_text(gen)
                ref = normalize_text(ref)
                bleu = compute_bleu(gen, ref)
                rouge = compute_rouge_l(gen, ref)
                bert_f1 = compute_bertscore(gen, ref)
                bleu_scores.append(bleu)
                rouge_scores.append(rouge)
                bert_scores.append(bert_f1)
                sample_outputs.append((gen, ref))
            del input_ids, attention_mask, generated_ids
            torch.cuda.empty_cache()
    avg_bleu = np.mean(bleu_scores)
    avg_rouge = np.mean(rouge_scores)
    avg_bert = np.mean(bert_scores)
    print(f"T5 {task} Evaluation:")
    print(f"Average BLEU: {avg_bleu:.4f}")
    print(f"Average ROUGE-L: {avg_rouge:.4f}")
    print(f"Average BERTScore F1: {avg_bert:.4f}")
    print(f"Sample Outputs (First 5) for {task}:")
    for gen, ref in sample_outputs[:5]:
        print(f"Generated: {gen}")
        print(f"Reference: {ref}\n")
    return avg_bleu, avg_rouge, avg_bert

# Evaluate T5 QA on QA and triple tasks
t5_qa_bleu, t5_qa_rouge, t5_qa_bert = evaluate_t5(t5_qa_model, qa_val_loader, task="qa")
t5_triple_bleu, t5_triple_rouge, t5_triple_bert = evaluate_t5(t5_qa_model, triple_val_loader, task="triple")

# Evaluate DPR on both QA and triple tasks at k=1, 5, 10
def evaluate_dpr_k(ctx_encoder, question_encoder, val_loader, candidates, k_values=[1, 5, 10], task: str = "qa"):
    ctx_encoder.eval()
    question_encoder.eval()
    mrr_k, precision_k = {k: [] for k in k_values}, {k: [] for k in k_values}
    eval_candidates = candidates
    print(f"Task: {task}, Using candidate pool size: {len(eval_candidates)}")
    candidate_inputs = ctx_tokenizer(eval_candidates, return_tensors="pt", padding=True, truncation=True, max_length=CONFIG.MAX_LENGTH)
    candidate_inputs = {k: v.to(CONFIG.DEVICE) for k, v in candidate_inputs.items()}
    with torch.no_grad():
        candidate_embeddings = ctx_encoder(**candidate_inputs).pooler_output
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Evaluating DPR {task}"):
            question_inputs = {
                "input_ids": batch["dpr_input_ids"].to(CONFIG.DEVICE),
                "attention_mask": batch["dpr_attention_mask"].to(CONFIG.DEVICE)
            }
            references = batch["answer"]
            question_embeddings = question_encoder(**question_inputs).pooler_output
            similarities = torch.matmul(question_embeddings, candidate_embeddings.T)
            rankings = torch.argsort(similarities, dim=1, descending=True)
            for i, (ranking, ref) in enumerate(zip(rankings, references)):
                ref_idx = eval_candidates.index(ref) if ref in eval_candidates else -1
                if ref_idx == -1:
                    continue
                for k in k_values:
                    top_k_indices = ranking[:k]
                    rank = (top_k_indices == ref_idx).nonzero(as_tuple=True)[0].item() + 1 if ref_idx in top_k_indices else len(eval_candidates)
                    mrr_k[k].append(1.0 / rank)
                    precision_k[k].append(1.0 if ref_idx in top_k_indices else 0.0)
            del question_inputs, similarities, rankings
            torch.cuda.empty_cache()
    results = {}
    for k in k_values:
        avg_mrr = np.mean(mrr_k[k])
        avg_precision = np.mean(precision_k[k])
        results[f"mrr_at_{k}"] = avg_mrr
        results[f"precision_at_{k}"] = avg_precision
        print(f"DPR Evaluation ({task}) at k={k}:")
        print(f"MRR@{k}: {avg_mrr:.4f}")
        print(f"Precision@{k}: {avg_precision:.4f}")
    return results

# Evaluate DPR on QA and triple tasks
dpr_qa_metrics = evaluate_dpr_k(ctx_encoder, question_encoder, qa_val_loader, all_candidates, task="qa")
dpr_triple_metrics = evaluate_dpr_k(ctx_encoder, question_encoder, triple_val_loader, all_candidates, task="triple")

# Evaluate Ensemble on both QA and triple tasks at k=1, 5, 10
def ensemble_evaluate_k(t5_model, ctx_encoder, question_encoder, val_loader, candidates, k_values=[1, 5, 10], top_k: int = 30, task: str = "qa"):
    print(f"Evaluating ensemble (T5 + DPR) for {task}...")
    t5_model.eval()
    ctx_encoder.eval()
    question_encoder.eval()
    mrr_k, precision_k = {k: [] for k in k_values}, {k: [] for k in k_values}
    candidate_inputs = ctx_tokenizer(candidates, return_tensors="pt", padding=True, truncation=True, max_length=CONFIG.MAX_LENGTH)
    candidate_inputs = {k: v.to(CONFIG.DEVICE) for k, v in candidate_inputs.items()}
    with torch.no_grad():
        candidate_embeddings = ctx_encoder(**candidate_inputs).pooler_output
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Ensemble Evaluating {task}"):
            input_ids = batch["t5_input_ids"].to(CONFIG.DEVICE)
            attention_mask = batch["t5_attention_mask"].to(CONFIG.DEVICE)
            references = batch["answer"]
            generated_ids = t5_model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=100,
                num_beams=30,
                temperature=0.5,
                num_return_sequences=top_k
            )
            generated_texts = [t5_tokenizer.decode(g_ids, skip_special_tokens=True).lower().strip() for g_ids in generated_ids]
            generated_texts = [generated_texts[i:i+top_k] for i in range(0, len(generated_texts), top_k)]
            for i, (gen_texts, ref) in enumerate(zip(generated_texts, references)):
                if not gen_texts:
                    continue
                gen_inputs = question_tokenizer(gen_texts, return_tensors="pt", padding=True, truncation=True, max_length=CONFIG.MAX_LENGTH)
                gen_inputs = {k: v.to(CONFIG.DEVICE) for k, v in gen_inputs.items()}
                gen_embeddings = question_encoder(**gen_inputs).pooler_output
                gen_similarities = torch.matmul(gen_embeddings, candidate_embeddings.T)
                rankings = torch.argsort(gen_similarities, dim=1, descending=True)
                if rankings.size(0) == 0 or rankings.size(1) == 0:
                    continue
                ref_idx = gen_texts.index(ref) if ref in gen_texts else -1
                if ref_idx == -1:
                    continue
                for k in k_values:
                    top_k_indices = rankings[:, :k]
                    rank = (top_k_indices == ref_idx).nonzero(as_tuple=True)[0].item() + 1 if ref_idx in top_k_indices else len(gen_texts)
                    mrr_k[k].append(1.0 / rank)
                    precision_k[k].append(1.0 if ref_idx in top_k_indices else 0.0)
            del input_ids, attention_mask, generated_ids, gen_inputs, gen_similarities, rankings
            torch.cuda.empty_cache()
    results = {}
    for k in k_values:
        avg_mrr = np.mean(mrr_k[k]) if mrr_k[k] else 0.0
        avg_precision = np.mean(precision_k[k]) if precision_k[k] else 0.0
        results[f"mrr_at_{k}"] = avg_mrr
        results[f"precision_at_{k}"] = avg_precision
        print(f"Ensemble (T5 + DPR) Evaluation ({task}) at k={k}:")
        print(f"MRR@{k}: {avg_mrr:.4f}")
        print(f"Precision@{k}: {avg_precision:.4f}")
    return results

# Evaluate ensemble on QA and triple tasks
ensemble_qa_metrics = ensemble_evaluate_k(t5_qa_model, ctx_encoder, question_encoder, qa_val_loader, all_candidates, task="qa")
ensemble_triple_metrics = ensemble_evaluate_k(t5_qa_model, ctx_encoder, question_encoder, triple_val_loader, all_candidates, task="triple")

# Save quantitative results
quantitative_results = {
    "t5_qa": {"bleu": t5_qa_bleu, "rouge": t5_qa_rouge, "bertscore": t5_qa_bert},
    "t5_triple": {"bleu": t5_triple_bleu, "rouge": t5_triple_rouge, "bertscore": t5_triple_bert},
    "dpr_qa": dpr_qa_metrics,
    "dpr_triple": dpr_triple_metrics,
    "ensemble_qa": ensemble_qa_metrics,
    "ensemble_triple": ensemble_triple_metrics
}

quantitative_path = os.path.join(CONFIG.BASE_PATH, "quantitative_results.json")
with open(quantitative_path, "w") as f:
    json.dump(quantitative_results, f)
print(f"Saved quantitative results at {quantitative_path}")

Evaluating T5 for qa...


Evaluating qa:   0%|          | 0/7 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFun

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

...

DPR Evaluation (triple) at k=1:
MRR@1: 0.0075
Precision@1: 0.0070
DPR Evaluation (triple) at k=5:
MRR@5: 0.0102
Precision@5: 0.0140
DPR Evaluation (triple) at k=10:
MRR@10: 0.0112
Precision@10: 0.0220
Evaluating ensemble (T5 + DPR) for qa...


Ensemble Evaluating qa: 100%|██████████| 7/7 [01:22<00:00, 11.76s/it]


Ensemble (T5 + DPR) Evaluation (qa) at k=1:
MRR@1: 0.0333
Precision@1: 0.0000
Ensemble (T5 + DPR) Evaluation (qa) at k=5:
MRR@5: 0.0333
Precision@5: 0.0000
Ensemble (T5 + DPR) Evaluation (qa) at k=10:
MRR@10: 0.0333
Precision@10: 0.0000
Evaluating ensemble (T5 + DPR) for triple...


Ensemble Evaluating triple: 100%|██████████| 63/63 [12:12<00:00, 11.62s/it]

Ensemble (T5 + DPR) Evaluation (triple) at k=1:
MRR@1: 0.0000
Precision@1: 0.0000
Ensemble (T5 + DPR) Evaluation (triple) at k=5:
MRR@5: 0.0000
Precision@5: 0.0000
Ensemble (T5 + DPR) Evaluation (triple) at k=10:
MRR@10: 0.0000
Precision@10: 0.0000
Saved quantitative results at /content/drive/MyDrive/LJMU-Datasets/quantitative_results.json


In [ ]:
# Qualitative Analysis (Human Assessment)

# Select samples for qualitative analysis (5 QA, 5 triple)
qa_samples = qa_val_df.sample(n=5, random_state=42)
triple_samples = triple_val_df.sample(n=5, random_state=42)

# Verify DataFrame structure
print(f"QA Samples Columns: {qa_samples.columns.tolist()}")
print(f"Triple Samples Columns: {triple_samples.columns.tolist()}")

# Generate predictions for qualitative analysis
def generate_predictions(model, val_loader, task: str = "qa"):
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in val_loader:
            # Debug: Print batch keys
            print(f"Batch keys: {batch.keys()}")
            input_ids = batch["t5_input_ids"].to(CONFIG.DEVICE)
            attention_mask = batch["t5_attention_mask"].to(CONFIG.DEVICE)
            references = batch["answer"]
            questions = batch["question"]
            generated_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=100,
                num_beams=20,
                temperature=0.5,
                no_repeat_ngram_size=2
            )
            generated_texts = [t5_tokenizer.decode(g_ids, skip_special_tokens=True).lower().strip() for g_ids in generated_ids]
            for q, gen, ref in zip(questions, generated_texts, references):
                predictions.append({"question": q, "generated": gen, "reference": ref})
            del input_ids, attention_mask, generated_ids
            torch.cuda.empty_cache()
    return predictions

# Create DataLoaders with adjusted batch size
qa_sample_loader = DataLoader(
    RetrievalDataset(qa_samples, t5_tokenizer, question_tokenizer, task="qa", candidate_objects=all_candidates),
    batch_size=1,  # Adjusted to handle small sample size
    shuffle=False,
    num_workers=CONFIG.NUM_WORKERS
)
triple_sample_loader = DataLoader(
    RetrievalDataset(triple_samples, t5_tokenizer, question_tokenizer, task="triple", candidate_objects=all_candidates),
    batch_size=1,  # Adjusted to handle small sample size
    shuffle=False,
    num_workers=CONFIG.NUM_WORKERS
)

# Generate predictions for QA and triple samples
qa_predictions = generate_predictions(t5_qa_model, qa_sample_loader, task="qa")
triple_predictions = generate_predictions(t5_qa_model, triple_sample_loader, task="triple")

# Perform qualitative analysis
qualitative_analysis = {
    "qa": [],
    "triple": []
}

# Assess QA predictions
for pred in qa_predictions:
    question = pred["question"]
    generated = pred["generated"]
    reference = pred["reference"]
    coherence = "Coherent" if len(generated.split()) > 3 and generated != reference else "Incoherent"
    reliability = "Reliable" if generated == reference else "Unreliable"
    interpretability = "Interpretable" if len(generated.split()) > 3 else "Not Interpretable"
    qualitative_analysis["qa"].append({
        "question": question,
        "generated": generated,
        "reference": reference,
        "coherence": coherence,
        "reliability": reliability,
        "interpretability": interpretability
    })

# Assess triple predictions
for pred in triple_predictions:
    question = pred["question"]
    generated = pred["generated"]
    reference = pred["reference"]
    coherence = "Coherent" if len(generated.split()) > 1 and generated != reference else "Incoherent"
    reliability = "Reliable" if generated == reference else "Unreliable"
    interpretability = "Interpretable" if len(generated.split()) > 1 else "Not Interpretable"
    qualitative_analysis["triple"].append({
        "question": question,
        "generated": generated,
        "reference": reference,
        "coherence": coherence,
        "reliability": reliability,
        "interpretability": interpretability
    })

# Save qualitative analysis
qualitative_path = os.path.join(CONFIG.BASE_PATH, "qualitative_analysis.json")
with open(qualitative_path, "w") as f:
    json.dump(qualitative_analysis, f)
print(f"Saved qualitative analysis at {qualitative_path}")

# Print sample qualitative analysis
print("Sample Qualitative Analysis (QA):")
for entry in qualitative_analysis["qa"][:2]:
    print(f"Question: {entry['question']}")
    print(f"Generated: {entry['generated']}")
    print(f"Reference: {entry['reference']}")
    print(f"Coherence: {entry['coherence']}")
    print(f"Reliability: {entry['reliability']}")
    print(f"Interpretability: {entry['interpretability']}\n")

print("Sample Qualitative Analysis (Triple):")
for entry in qualitative_analysis["triple"][:2]:
    print(f"Question: {entry['question']}")
    print(f"Generated: {entry['generated']}")
    print(f"Reference: {entry['reference']}")
    print(f"Coherence: {entry['coherence']}")
    print(f"Reliability: {entry['reliability']}")
    print(f"Interpretability: {entry['interpretability']}\n")

QA Samples Columns: ['question', 'context', 'answer']
Triple Samples Columns: ['question', 'context', 'answer']
Batch keys: dict_keys(['task', 't5_input_ids', 't5_attention_mask', 't5_labels', 'dpr_input_ids', 'dpr_attention_mask', 'question', 'answer'])
Batch keys: dict_keys(['task', 't5_input_ids', 't5_attention_mask', 't5_labels', 'dpr_input_ids', 'dpr_attention_mask', 'question', 'answer'])
Batch keys: dict_keys(['task', 't5_input_ids', 't5_attention_mask', 't5_labels', 'dpr_input_ids', 'dpr_attention_mask', 'question', 'answer'])
Batch keys: dict_keys(['task', 't5_input_ids', 't5_attention_mask', 't5_labels', 'dpr_input_ids', 'dpr_attention_mask', 'question', 'answer'])
Batch keys: dict_keys(['task', 't5_input_ids', 't5_attention_mask', 't5_labels', 'dpr_input_ids', 'dpr_attention_mask', 'question', 'answer'])
Batch keys: dict_keys(['task', 't5_input_ids', 't5_attention_mask', 't5_labels', 'dpr_input_ids', 'dpr_attention_mask', 'question', 'answer', 'label_idx'])
Batch keys: dict_